# 数据获取和存储

+ 在这一节中，我们主要介绍如何获取和读入数据，以及如何存储数据。
+ 其中涉及数据格式，网络数据抓取等内容。
+ 程序在Python 2.7运行，如果在3.3或更高版本中，可能会有所差别，特别是几个涉及中文字符的例子。

+ 下面是基本设定，需要导入的包和一些简单设定。

In [1]:
from __future__ import division
import os
import sys

import numpy as np
from numpy.random import randn
from pandas import Series, DataFrame
import pandas as pd
import matplotlib.pyplot as plt


np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4)

## Reading and Writing Data in Text Format

+ 读写文本格式的数据， 在所有pandas 包中关于读取的函数中个，read_csv 和read_table 是使用最多的。
+ 在读取存储在文本文件中的数据时，首先要关注一下 文档路径 以及 当前 notebook的工作目录

In [2]:
%pwd

u'/Users/wrk/Documents/notebook0'

+ 打开文件，大致了解一下文档中数据的格式

In [3]:
#运用dos命令
!type data/ex1.csv    ##linux -->?

data/ex1.csv is data/ex1.csv


+ 逗号分隔的数据，可以用read_csv函数读入

In [4]:
df = pd.read_csv('data/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


+ 也可以用read_table读入，只是这时需要 设置 数据之间的分隔符参数： sep=','

In [5]:
pd.read_table('data/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


+ 看看ex2文件，其中第一行不是变量名

In [6]:
!type data\ex2.csv

/bin/sh: line 0: type: dataex2.csv: not found


+ 因此需要指定和变量名相关的参数：header=None 或者 names=['a', 'b', 'c', 'd', 'message'] 等

In [7]:
pd.read_csv('data/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
pd.read_csv('data/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


+ 我们也可以指定DataFrame 的 行索引关键字：index_col='message'  说明用message这列作为行索引关键字
+ 也可以用index_col=4,message的列号指定其为关键字

In [11]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('data/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


+ 再看下面的例子

In [14]:
!type data\csv_mindex.csv

/bin/sh: line 0: type: datacsv_mindex.csv: not found


+ index关键字可以是两列，这样就有多层索引。

In [15]:
parsed = pd.read_csv('data/csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

+ 分隔符可以有多种形式，比如如果分隔符为数量不等长空格符，则用正则表达式 '\s+'表示分隔符格式

In [16]:
list(open('data/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [21]:
result = pd.read_table('data/ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


+ pandas自动推断出第一行为列名，第一列为index. 

+ 有时候文档中会有说明文字，需要指定哪些行不用读入。
+ 选项 skiprows给出跳过的行。

In [14]:
!type data/ex4.csv

data/ex4.csv is data/ex4.csv


In [24]:
pd.read_csv('data/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


+ pandas会自动处理常见的丢失值，不过也可以指定哪些值是缺失值
+ 我们甚至可以给不同列指定不同的缺失值。
+ 下面的例子中有标注了的缺失值 NA；也有没有标注的，连续两个逗号表示中间缺了一个数

In [25]:
!type data\ex5.csv

/bin/sh: line 0: type: dataex5.csv: not found


In [26]:
result = pd.read_csv('data/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


+ 指定缺失值的实例

In [29]:
result = pd.read_csv('data/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [30]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
print(sentinels)
pd.read_csv('data/ex5.csv', na_values=sentinels)

{'message': ['foo', 'NA'], 'something': ['two']}


,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


+ 处理带中文的表格，用Python 3.3的同学看看这里是否有差别

In [33]:
Astock=pd.read_table("data/A.txt",encoding="gbk")

Astock.columns 

Index([         u'代码',          u'名称',        u'涨幅%%',          u'涨跌',
              u'最高%%',          u'现价',        u'开盘%%',          u'现量',
              u'投资收益',         u'市现率',         u'市净率',       u'市盈(动)',
             u'每股未分配',     u'每股未分配.1',          u'昨收',     u'资产负债率%%',
              u'人均持股',      u'净利润率%%',        u'细分行业',     u'流通股本(万)',
                u'地区',        u'振幅%%', u'Unnamed: 22'],
      dtype='object')

### 逐块读入数据

当文件很大的时候，比如上百M。可能逐块读入更好。下面的例子有10000行数据

In [34]:
result=pd.read_csv('data/ex6.csv')
pd.read_csv('data/ex6.csv', nrows=5) #只读前5行

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


+ 设置 chunksize实现逐块阅读
+ 我们可以通过.read　方法将前面若干行读出来。

In [40]:
chunker = pd.read_csv('data/ex6.csv', chunksize=1000)
chunker.read(5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


+ 也可以用循环分块处理
+ 下面例子统计key列中各字母出现的频数，这里用到了Series求和的对齐功能。
+ .value_counts()是统计频数的方法

In [42]:
chunker = pd.read_csv('data/ex6.csv', chunksize=1000)

tot = Series([])

for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.order(ascending=False)
tot[:5]

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: order is deprecated, use sort_values(...)


E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
dtype: float64

### 将数据写出到文本格式

首先看看前面处理过的csv文件

In [43]:
data = pd.read_csv('data/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


+ 用数据data的方法to_csv写出到文件

In [45]:
data.to_csv('data/out.csv')
out = pd.read_csv('data/out.csv')
out

,Unnamed: 0,something,a,b,c,d,message
0,0,one,1,2,3.0,4,NaN
1,1,two,5,6,NaN,8,world
2,2,three,9,10,11.0,12,foo


+ sys.stdout 是输出到屏幕，看先观察一下效果
+ sep='|' 分割符为 | 

In [46]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


+ na_rep='NULL' 表示用"NULL"代替 缺失值位置

In [48]:
data.to_csv(sys.stdout, na_rep='FUCK')

,something,a,b,c,d,message
0,one,1,2,3.0,4,FUCK
1,two,5,6,FUCK,8,world
2,three,9,10,11.0,12,foo


+  index=False, header=False 说明不输出行和列索引

In [49]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


+ 当然也可以重新指定列名和索引

In [53]:
print(data)
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'],na_rep='Fuck')

  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo
a,b,c
1,2,3.0
5,6,Fuck
9,10,11.0


+ 下面是一个时间序列索引数据读写，我们会在后面专门介绍

In [59]:
dates = pd.date_range('1/1/2000', periods=7)
print(dates)
ts = Series(np.arange(7), index=dates)
print(ts)
ts.to_csv('data/tseries.csv')
a = pd.read_csv('data/tseries.csv')
print(a)

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')
2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int64
   2000-01-01  0
0  2000-01-02  1
1  2000-01-03  2
2  2000-01-04  3
3  2000-01-05  4
4  2000-01-06  5
5  2000-01-07  6


In [62]:
Series.from_csv('data/tseries.csv', parse_dates=True)

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

### 从网络url中读

+ 网络url直接指向一个文本格式的文件：一个csv文件或者一个txt文件
+ 我们以从sina下载 高频数据，日线数据为例进行说明，这里要用到 urllib包。
+ 我们用 urlretrieve 函数读入存在本地，然后用read_table 读入数据
+ 可以直接用read_table读网络rul指向的文本格式文件，
   -  data= pd.read_table(url,na_values="--",encoding="gbk",parse_dates=True,index_col=0)
   -  不过出现问题，原因不明。


In [ ]:
import urllib 
url = 'http://market.finance.sina.com.cn/downxls.php?date=2016-09-09&symbol=sh600540'  
urllib.urlretrieve(url, "demo.txt")
dataf= pd.read_table("demo.txt",na_values="--",encoding="gbk",parse_dates=True,index_col=0)
print(dataf[:3])
 

注意数据中没有日期，read_table自己加上的

### 读取 网络表格，

+ 很多网络数据都是以表格形式存在的，我们需要分析html文件，提取其中的表格
+  导入包和表格所在的url地址
```
from lxml.html import parse
from urllib2 import urlopen
#url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_MarketHistory/stockid/601899.phtml?year=2015&jidu=1'
url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_FuQuanMarketHistory/stockid/601899.phtml?year=2015&jidu=1'
```
+ 读取并提取所有表格
```
parsed = parse(urlopen(url))
doc = parsed.getroot()
tables=doc.findall('.//table')
```

+ 选定我们需要的表格

```
table = tables[19] 
```

+  找到表格中所有行

```
rows = table.findall('.//tr')

```
+ 网络表格一般都有标题行，即th单元格，数据行则是td单元格。用如下的函数提取数据单元中的数据
```
def _unpack(row, kind='td'):
    elts = row.findall('.//%s' % kind)
    return [val.text_content().strip() for val in elts] 
```

如果kind ="th"，则是标题行中的数据，相当于变量名。

+ 需要首先观察网页中的表格，必要时需要看html原文件（确定所需表格是第几个表格是一定要看原文件），看看相应数据从第几行开始，然后将所有数据整合在一起生成DataFrame数据。

   - 读入第三行开始的所有行，生成一个list的list，其中内层的list代表一行数据
  
  ``` 
    data = [_unpack(r) for r in rows[2:]] 
   ```
   - 也可以用下面代码读入中文列名(变量名)
   
  ```
   header=_unpack(row[1],kind="td")
   ```
   - 整合为 DataFrame数据
   
   ```
   TextParser(data, names=header).get_chunk()
   ```

In [ ]:
from lxml.html import parse
from urllib2 import urlopen
#url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_MarketHistory/stockid/601899.phtml?year=2015&jidu=1'
url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_FuQuanMarketHistory/stockid/601899.phtml?year=2015&jidu=1'
parsed = parse(urlopen(url))
doc = parsed.getroot()
tables=doc.findall('.//table')
table = tables[19]
#rows = calls.findall('.//tr')
from pandas.io.parsers import TextParser
def _unpack(row, kind='td'):
    elts = row.findall('.//%s' % kind)
    return [val.text_content().strip() for val in elts] # .strip()去掉\r\t\n之类的字符
def parse_options_data(table):
    rows = table.findall('.//tr')
    header = ['date','open','high','close','low','vol','amount','ratio'] # 回避中文处理
    data = [_unpack(r) for r in rows[2:]]
    return TextParser(data, names=header).get_chunk()
datad = parse_options_data(table)

In [ ]:
datad[:5]

+ 其它解析html的例子，比如获得链接地址，在html中链接的标签是a
+ 获得具体链接字符串得用 .get 方法

In [68]:
links=doc.findall('.//a')
lin=links[0]
lin.get('href')

'http://www.sina.com.cn/'

In [69]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[-5:]

['http://www.sina.com.cn/intro/lawfirm.shtml',
 'http://english.sina.com',
 'http://members.sina.com.cn/apply/',
 'http://help.sina.com.cn/',
 'http://www.sina.com.cn/intro/copyright.shtml']

+   .text_content方法 可以得到标签内的文本

In [70]:
cont=links[10].text_content() 
print(cont)
lin=links[10].get('href') 
print(lin)

基金
http://finance.sina.com.cn/fund/index.shtml


## 二进制数据 格式  Binary data formats

+ Python 数据的二进制存储文档一般是带有 .pkl 扩展名的文件。 
+ 在pandas中，有读入数据的方法，而数据对象对象DataFrame和Series也有将数据写入二进制文件的方法。
+ 在Python中有 pickle 包中的 load 和dump 函数进行数据的读和写。

In [71]:
frame = pd.read_csv('data/ex1.csv')
frame
frame.to_pickle('data/frame_pickle')

In [72]:
pd.read_pickle('data/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


+ 在Python中有 pickle 包中的 load 和dump 函数进行数据的读和写。

In [73]:
import pickle
file1=open("data/pickle_dump.pkl","w")
pickle.dump(frame,file=file1)
file1.close()
file1=open("data/pickle_dump.pkl","r")
data1=pickle.load(file=file1)
file1.close()
!del data\pickle_dump.pkl  
data1

/bin/sh: del: command not found


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


+ 文件必须关掉后才可以删除

## 数据库使用

+ 使用文本文件进行存储和读取的效率比较低，当数量两非常大时，需要考虑使用数据库。
+ 下面介绍使用SQLite数据库，其通过python的sqlite3驱动
+ python和数据库之间的交互非常简单，特别是pandas提供了一些函数，使我们能轻松进行数据库操作

命令|功能
------|------
import sqlite3 |导入数据库包
conn = sqlite3.connect('data/mydatabase.db')| 创建在硬盘上面
conn = sqlite3.connect('"memory:') | 创建在内存上面

+ 其中conn对象是数据库链接对象，而对于数据库链接对象来说，具有以下操作：

命令|功能
------|------
        commit()     |       事务提交
        rollback()    |     事务回滚
        close()       |     关闭一个数据库链接
        cursor()      |    创建一个游标
      cu = conn.cursor() |   这样我们就创建了一个游标对象：cu. 在sqlite3中，所有sql语句的执行都要在游标对象的参与下完成      
    对于游标对象cu，具有以下具体操:|
        execute()        |执行一条sql语句
        executemany()   |执行多条sql语句
        close()         |游标关闭
        fetchone()     |从结果中取出一条记录
        fetchmany()    |从结果中取出多条记录
        fetchall()   |从结果中取出所有记录
        scroll()   |游标滚动    
   

+ 下面的例子在内存中创建了一个数据库，并生成数据库的一个表

In [74]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

+ 将数据插入相应的表,使用sqlite中的函数

In [75]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

+ 我们可以使用pandas中的函数向数据库中插入一个表格

         import pandas.io.sql as sql

+ 在python2.7中，我们插入有中文关键字的DataFrame时需要
```
reload(sys)
sys.setdefaultencoding( "utf-8" )
```
+ to_sql函数可以插入一个DataFrame表格，注意如果表格存在，需要设置参数if_exists
    -  if_exists='fail' 如果存在就放弃插入
    -  if_exists='replace' 删掉原来的，创建新的表 
    -  if_exists='append' 插入数据

In [76]:
import pandas.io.sql as sql
reload(sys)
sys.setdefaultencoding( "utf-8" )
sql.to_sql(dataf,"highfreq",con)
sql.to_sql(datad,"daily",con)

+ 从数据库中读取数据，用到了pandas中的函数

In [77]:
mysql="SELECT * from highfreq LIMIT 5"
sql.read_sql(mysql,con)

,成交时间,成交价,价格变动,成交量(手),成交额(元),性质
0,2016-10-04 15:00:00,7.27,NaN,0,0,卖盘
1,2016-10-04 15:00:00,7.27,-0.01,0,0,卖盘
2,2016-10-04 14:59:57,7.28,0.01,502,365456,买盘
3,2016-10-04 14:59:51,7.27,NaN,21,15267,卖盘
4,2016-10-04 14:59:46,7.27,NaN,175,127224,卖盘


+ index_col 参数指定index

In [45]:

sql.read_sql('select * from daily limit 5', con,index_col=["index","date"])


,,open,high,close,low,vol,amount,ratio
index,date,,,,,,,
0,2015-03-31,7.712,7.712,7.504,7.486,489626016,2139222144,1.737
1,2015-03-30,7.660,7.799,7.729,7.469,597394304,2610106880,1.737
2,2015-03-27,7.747,7.868,7.695,7.660,469180672,2088988800,1.737
3,2015-03-26,7.834,7.834,7.660,7.486,711915328,3128761344,1.737
4,2015-03-25,7.799,8.094,7.851,7.729,1436268672,6534120448,1.737



## 函数

编一个从sina抓取特定时间段内的日线数据的函数。注意和上面的细节上的不同。
 1. 股票复权与否
 2. 指数还是股票
 3. 复权数据多一列复权因子

In [46]:
from lxml.html import parse
from urllib2 import urlopen
import pandas as pd


from lxml.html import parse
from urllib2 import urlopen
import pandas as pd

def get_sina_stock(stockid,start,end,isstock=True):    
    def _unpack(row, kind='td'):
        elts = row.findall('.//%s' % kind)
        return [val.text_content().strip() for val in elts] # .strip()去掉\r\t\n之类的字符    
    def parse_options_data(table,isstock):
        rows = table.findall('.//tr')
        data = [_unpack(r) for r in rows[2:]]
        if isstock:
            colnames = ['date','open','high','close','low','vol','amount','ratio'] # 回避中文处理
        else:
            colnames = ['date','open','high','close','low','vol','amount']
        data= pd.DataFrame(data,columns=colnames)
        data.index=pd.to_datetime(data["date"])
        data=data.drop("date",axis=1)
        data=data.ix[range(len(data)-1,-1,-1)]
        return  data
    data=pd.DataFrame()
    daterange=pd.date_range(start,end,freq="Q")
    daterange=daterange.insert(len(daterange),daterange[-1]+1)
    for cq in daterange: 
        if isstock:
            url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_FuQuanMarketHistory/stockid/'+str(stockid)+'.phtml?year='+str(cq.year)+'&jidu='+str(cq.month/3)
        else:            
            url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_MarketHistory/stockid/'+str(stockid)+'/type/S.phtml?year='+str(cq.year)+'&jidu='+str(cq.month/3)
        try:
            parsed = parse(urlopen(url))
        except:
            print "download failed for year=" +str(cq.year)+',  jidu='+str(cq.month/3)
            continue
        doc = parsed.getroot()
        tables=doc.findall('.//table')     
        da= tables[-1] # last table 19 for fuquan data and 4 for index
        datatem = parse_options_data(da,isstock)
        data=pd.concat([data,datatem])
    return pd.DataFrame(data[start:end],dtype=float)    


In [47]:
data1=get_sina_stock(601899,"1990-1-1","2015-8-15")
data2=get_sina_stock("000001","1990-1-1","2015-8-15",isstock=False)
data2.to_csv("data/000001.csv")
pd.concat([data1.head(3),data1.tail(3)])

,open,high,close,low,vol,amount,ratio
date,,,,,,,
2008-04-25,9.980,22.000,13.920,9.610,971534170,10346791205,1.000
2008-04-28,12.530,12.530,12.530,12.530,9192800,115185784,1.000
2008-04-29,11.280,11.280,11.280,11.280,7794300,87919704,1.000
2015-08-12,8.169,8.753,8.346,7.921,1085489408,5061476352,1.772
2015-08-13,8.151,8.417,8.240,7.974,588221952,2721628928,1.772
2015-08-14,8.151,8.186,8.009,7.991,442381184,2012518528,1.772


In [48]:
pd.concat([data2.head(3),data2.tail(3)])

,open,high,close,low,vol,amount
date,,,,,,
1990-12-19,96.050,99.980,99.980,95.790,126000,4.940000e+05
1990-12-20,104.300,104.390,104.390,99.980,19700,8.400000e+04
1990-12-21,109.070,109.130,109.130,103.730,2800,1.600000e+04
2015-08-12,3881.228,3937.772,3886.320,3871.136,44268828800,5.970503e+11
2015-08-13,3869.911,3955.789,3954.556,3838.159,43007331200,5.786855e+11
2015-08-14,3976.405,4000.684,3965.335,3939.835,46798822400,6.474665e+11



## 分组作业

+ 4-6人一组
+ 每组选择一个行业，或者一个板块的所有股票，编程完成下面任务：
   - 从sina下载近5年的日线数据，将所有数据存为pandas的panel数据（复权和不复权各一个数据文件）
   - 从sina下载一周高频分时数据，将所有数据存为pandas的panel数据
   - 选择 （ 财务摘要，财务指标，资产负债表，利润表，或现金流量表）中的一个或多个表 近5年的数据，下载行业内所有股票构建一个Sqlite数据库    
     -  比如打开
        http://vip.stock.finance.sina.com.cn/corp/go.php/vFD_FinanceSummary/stockid/600030/displaytype/4.phtml ，
        在"财务数据"后面就可以看见相应的链接


     
